In [204]:
import pandas as pd
import numpy as np
import math
import ast
import re
from nltk.tokenize import word_tokenize

df = pd.read_csv('../data/redfin.csv')
df.shape
df.head()
df = df.rename(columns={'baths':'Baths', 
                   'beds':'Beds', 
                   'price': 'Price', 
                   'sqft':'SqFt',
                   'street':'Address',
                   'zipcode':'Zip'})
df.shape

(13662, 10)

In [205]:
# Stats details 
index = range(0, df.shape[0])
stats_df = pd.DataFrame(index = index, columns = ['Price', 'Beds', 'SqFt', 'Baths', 'Address'])
for column in stats_df.columns.drop('Address'):
    data = [''.join(re.findall(r'[0-9.]', str(i))) for i in df[column]]
    stats_df[column] = data
stats_df['Address'] = df['Address']
for count, zipcode in enumerate(df.Zip):
    if len(zipcode.split(','))==2:
        zipcode = re.sub(' - Century City','',zipcode)
        zipcode = re.sub('County -','',zipcode)
        City = [i.capitalize() for i in word_tokenize(zipcode.split(',')[0])]
        City = ' '.join(City)
        stats_df.loc[count, 'City'] = City
        stats_df.loc[count, 'Zip'] = ''.join(re.findall('\d', zipcode.split(',')[1]))
    elif zipcode:
        stats_df.loc[count, 'Zip'] = ''.join(re.findall('\d', zipcode.split(',')[0]))
stats_df = stats_df[stats_df.Price.notna()]
stats_df

,Price,Beds,SqFt,Baths,Address,City,Zip
0,862173,4,1887,3,14281 Jacaranda Ln,Westminster,92683
1,533240,3,1547,3,2230 W Anacasa Way,Anaheim,92804
2,1165026,4,1988,2.5,29102 Mira Vis,Laguna Niguel,92677
3,,,,,139 TRUMPET FLOWER,Irvine,92618
4,1266320,3,1922,2.5,172 Costa Mesa St,Costa Mesa,92627
...,...,...,...,...,...,...,...
13657,2400000,5,4530,5.5,9691 Villa Woods Dr,Villa Park,92861
13658,1276000,4,3274,2.5,10432 Alma Ln,Villa Park,92861
13659,2175000,5,5219,4.5,9841 Ludwig St,Villa Park,92861
13660,1400000,5,3377,3,9601 Dodson Way,Villa Park,92861


In [206]:
## Key details 
key_df = pd.DataFrame(index = index)
for count, details in enumerate(df['key_details_dict']):
    dict_ = ast.literal_eval(details)
    for key, value in dict_.items():
        key = re.sub(r"[/]", 'Per', key)
        key = re.sub(r"[#\. ']", '', key)
        key_df.loc[count, key] = value
        
# Drop useless columns
drop_columns = ['BuyersBrokerageCommission', 'Style', 'Status', 'MLS','HOADues', 'Baths']
key_df = key_df.drop(drop_columns, axis=1)

# Clean Up Numerical Columns, "Lot Size" and "Baths" Columns Requires Special Adjustment
columns = key_df.columns.drop(['PropertyType','LotSize', 'Community'])
for column in columns:
    data = key_df[column].replace('/month','')
    data = [''.join(re.findall(r'[0-9.]', str(i))) for i in data]
    key_df[column] = data
    key_df[column].replace('', np.nan)
key_df

,PropertyType,YearBuilt,Community,LotSize,RedfinEstimate,PricePerSqFt
0,NaN,,NaN,NaN,,
1,NaN,,NaN,NaN,,
2,NaN,,NaN,NaN,,
3,NaN,,NaN,NaN,,
4,NaN,,NaN,NaN,,
...,...,...,...,...,...,...
13657,Single Family Residence,1970,73 - Villa Park,0.44 Acres,2369106,530
13658,Single Family Residence,1975,73 - Villa Park,0.48 Acres,1335412,390
13659,Single Family Residence,1974,73 - Villa Park,0.47 Acres,2268368,417
13660,Single Family Residence,1961,73 - Villa Park,0.55 Acres,1497148,415


In [207]:
## Helper function:
def convert_acre_to_sqft(lot, text):
    if lot[1].find('Acre')>-1:
        text = round(text * 43560)
    return text
            
# Clean Lot Size
for count, lot in enumerate(key_df['LotSize']):
    if str(lot) != 'nan':
        lot = str(lot)
        lot = lot.split(' ')
        text = re.findall(r'\b[0-9,.]+\b', lot[0])
        text = ''.join(text)
        text = text.replace(',','')
        text = float(text)
        text = convert_acre_to_sqft(lot, text)
        key_df.loc[count, 'LotSize'] = text

In [208]:
for column in key_df.columns:
    key_df[column] = pd.to_numeric(key_df[column], errors = 'ignore')
num_cols = key_df.select_dtypes(exclude='object').columns
num_cols

Index(['YearBuilt', 'LotSize', 'RedfinEstimate', 'PricePerSqFt'], dtype='object')

In [209]:
categ_cols = key_df.columns.drop(num_cols)
categ_cols

Index(['PropertyType', 'Community'], dtype='object')

In [210]:
key_df.PropertyType = [''.join(re.findall("[A-Za-z0-9/\- ]", str(i).lstrip())) for i in key_df.PropertyType]
#key_df.dropna(how='any',inplace=True)
sorted(key_df.PropertyType.unique())

['All Other Attached',
 'Apartment/Condo',
 'Commercial/Residential',
 'Condo/Co-op',
 'Condominium',
 'Condominium Residential Condo/Co-Op',
 'Detached',
 'Duplex',
 'Quadruplex',
 'Residential Single-Family',
 'Single Family Detached',
 'Single Family Home',
 'Single Family Residence',
 'Single Family Residential',
 'Single Family Residential Single-Family',
 'Townhome',
 'Townhouse',
 'Townhouse Residential Condo/Co-Op',
 'Townhouse Residential Single-Family',
 'Triplex',
 'nan']

In [211]:
# Property Type
PropertyType_dict = {'nan':None,
                     'Quadruplex':'Condominium',
                     'All Other Attached':'Condominium',
                     'Duplex':'Condominium',
                     'Single Family Residential Single-Family':'Single Family Residence',
                     'Townhouse Residential Condo/Co-Op':'Townhouse',
                     'Condominium Residential Condo/Co-Op':'Condominium',
                     'Residential Single-Family':'Single Family Residence',
                     'Apartment/Condo':'Condominium', 
                     'Townhome':'Townhouse',
                     'Condo/Co-op':'Condominium',
                     'Townhouse Residential Single-Family':'Single Family Residence',
                     'Single Family Detached':'Single Family Residence',
                     'Detached':'Single Family Residence',
                     'Single Family Home':'Single Family Residence', 
                     'Triplex':'Condominium',
                     'Single Family Residential':'Single Family Residence'}
key_df['PropertyType'].replace(PropertyType_dict, inplace=True)
key_df = key_df.loc[key_df.PropertyType!='Commercial/Residential']
key_df.PropertyType.unique()

array([None, 'Condominium', 'Single Family Residence', 'Townhouse'],
      dtype=object)

In [212]:
prop_df = pd.DataFrame(index=index)
replace_dict = {'1/2':'Half', '1/4':'Quarter', '3/4':'ThreeQuarters'}
for count, prop in enumerate(df['property_details_dict']):
    dict_ = ast.literal_eval(prop)    
    for key, value in dict_.items():
        
        # Latitude
        if re.findall(r'\bLatitude\b', str(key)):
            prop_df.loc[count, 'Latitude'] = value
            
        # Longitude
        if re.findall(r'\bLongitude\b', str(key)):
            prop_df.loc[count, 'Longitude'] = value  
        
        # Levels
        if re.match(r'\bLevels?\b', key):
            if re.findall('Multi/Split', str(value)):
                prop_df.loc[count, 'Levels'] = 'Multi/Split'
            elif re.findall('Three Or More', str(value)):
                prop_df.loc[count, 'Levels'] = 'Three Or More'
            elif re.findall('Two', str(value)):
                prop_df.loc[count, 'Levels'] = 'Two'
            elif re.findall('One', str(value)) or re.findall('1', str(value)):
                prop_df.loc[count, 'Levels'] = 'One'
        elif re.findall(r'\bMulti/Split\b',key):
            prop_df.loc[count, 'Levels']='Multi/Split'
        elif re.findall(r'\bThree Or More Levels?\b',key):
            prop_df.loc[count, 'Levels']='Three Or More'
        elif re.findall(r'\bTwo Levels?\b',key):
            prop_df.loc[count, 'Levels']='Two'
        elif re.findall(r'\bOne Levels?\b',key):
            prop_df.loc[count, 'Levels']='One'
            
        # Garage
        if re.findall('# of Garage', key):
            prop_df.loc[count, 'GarageSpaces']=value
        elif re.findall('4+ Car Garage', key):
            prop_df.loc[count, 'GarageSpaces']=4
        elif re.findall('3 Car Garage', key):
            prop_df.loc[count, 'GarageSpaces']=3
        elif re.findall('2 Car Garage', key):
            prop_df.loc[count, 'GarageSpaces']=2
        elif re.findall('Garage', key):
            prop_df.loc[count, 'GarageSpaces']=1
        
        # Common Walls
        if re.findall(r'\bCommon Walls\b', key):
            if re.findall('2+', str(value)):
                prop_df.loc[count, 'CommonWalls']='2+ Common Walls'
            elif re.findall('1', str(value)):
                prop_df.loc[count, 'CommonWalls']='1 Common Wall'
            elif re.findall('No Common Wall', str(value)):
                prop_df.loc[count, 'CommonWalls']='No Common Wall'
        
        # Parking
        if re.findall('#.*Parking', key):
            key = re.sub('#.*Parking.*', 'ParkingSpaces',key)
            prop_df.loc[count, key.strip()]=value
            
        # Spa
        if re.findall('Spa Features', key) or re.findall('Spa.*Description', key) or re.findall('Spa.*Construction', key):
            for sub_key in word_tokenize(value):
                if re.match('Private', str(sub_key)):
                    prop_df.loc[count, 'Spa'] = 'Private'
                elif re.match('Community', str(sub_key)) or re.match('Association', str(sub_key)):
                    prop_df.loc[count, 'Spa'] = 'Community'
        elif re.findall(r'\bSpa\b', key):
            for sub_key in word_tokenize(key):
                if re.match('Private', str(sub_key)):
                    prop_df.loc[count, 'Spa'] = 'Private'
                elif re.match('Community', str(sub_key)) or re.match('Association', str(sub_key)):
                    prop_df.loc[count, 'Spa'] = 'Community'
                    
        # Pool
        if re.findall('Pool Features', key) or re.findall('Pool.*Description', key) or re.findall('Pool.*Construction', key):
            for sub_key in word_tokenize(value):
                if re.match('Private', str(sub_key)):
                    prop_df.loc[count, 'Pool'] = 'Private'
                elif re.match('Community', str(sub_key)) or re.match('Association', str(sub_key)):
                    prop_df.loc[count, 'Pool'] = 'Community'
        elif re.findall(r'\bPool\b', key):
            for sub_key in word_tokenize(key):
                if re.match('Private', str(sub_key)):
                    prop_df.loc[count, 'Pool'] = 'Private'
                elif re.match('Community', str(sub_key)) or re.match('Association', str(sub_key)):
                    prop_df.loc[count, 'Pool'] = 'Community'
        
        # Kitchen Island
        if re.findall(r'\bKitchen Island\b', key):
            prop_df.loc[count, 'KitchenIsland'] = 'Yes'
                    
        # Patio
        if re.findall(r'\bPatio\b', key):
            prop_df.loc[count, 'Patio'] = 'Yes'
            
        # Main Level Bathrooms
        if re.findall(r'\bMain Level Bathrooms\b', key):
            prop_df.loc[count, 'MainLevelBathrooms']=value
             
        # Main Level Bedrooms
        if re.findall(r'\bMain Level Bedrooms\b', key):
            prop_df.loc[count, 'MainLevelBedrooms']=value
            
        # Walk-In Closet
        if re.findall(r'\bWalk.*Closets?\b', key):
            prop_df.loc[count, 'WalkInCloset'] = 'Yes'
            
        # High Ceiling/Cathedral
        if re.findall(r'\bCathedral\b', key) or re.findall(r'\bHigh Ceiling\b', key):
            prop_df.loc[count, 'HighCeiling'] = 'Yes'
            
        # Fire Place
        if re.findall(r'\bFireplace\b', key):
            prop_df.loc[count, 'Fireplace'] = 'Yes'
            
        # Central Air/Heating/Cooling
        if re.findall(r'\bCentral\b', key) and not (re.findall(r'\bCentral Vacuum\b', str(key)) or re.findall(r'\bVacuum Central\b', str(key))):
            prop_df.loc[count, 'CentralAir'] = 'Yes'
            
        # Outdoors (Hiking, Biking)
        if re.findall(r'\bHiking\b|\bBiking\b', key):
            prop_df.loc[count, 'CloseToOutdoors'] = 'Yes'
        elif re.findall(r'\bHiking\b|\bBiking\b', str(value)):
            prop_df.loc[count, 'CloseToOutdoors'] = 'Yes'
prop_df.columns

Index(['GarageSpaces', 'Fireplace', 'ParkingSpaces', 'CentralAir', 'Levels',
       'MainLevelBathrooms', 'MainLevelBedrooms', 'Spa', 'Patio', 'Pool',
       'CommonWalls', 'Latitude', 'Longitude', 'WalkInCloset', 'KitchenIsland',
       'CloseToOutdoors', 'HighCeiling'],
      dtype='object')

In [214]:
for col in prop_df.columns.drop(['MainLevelBathrooms','MainLevelBedrooms',
                                 'Latitude','Longitude',
                                 'Levels','CommonWalls']):
    prop_df[col] = pd.to_numeric(prop_df[col], errors='ignore')
    if prop_df[col].dtypes != 'object':
        prop_df[col] = prop_df[col].fillna(0)
    else:
        prop_df[col] = ['No' if str(i)=='nan' else i for i in prop_df[col]]
prop_df

,GarageSpaces,Fireplace,ParkingSpaces,CentralAir,Levels,MainLevelBathrooms,MainLevelBedrooms,Spa,Patio,Pool,CommonWalls,Latitude,Longitude,WalkInCloset,KitchenIsland,CloseToOutdoors,HighCeiling
0,1.0,No,0.0,No,NaN,NaN,NaN,No,No,No,NaN,NaN,NaN,No,No,No,No
1,1.0,No,0.0,No,NaN,NaN,NaN,No,No,No,NaN,NaN,NaN,No,No,No,No
2,1.0,Yes,0.0,No,NaN,NaN,NaN,No,No,No,NaN,NaN,NaN,No,No,No,No
3,0.0,No,0.0,No,NaN,NaN,NaN,No,No,No,NaN,NaN,NaN,No,No,No,No
4,1.0,No,0.0,No,NaN,NaN,NaN,No,No,No,NaN,NaN,NaN,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13657,3.0,Yes,3.0,Yes,Two,3,2,Private,No,Private,No Common Wall,33.82139600,-117.80743100,Yes,Yes,No,No
13658,3.0,Yes,3.0,Yes,Two,1,1,No,No,Private,No Common Wall,33.81090200,-117.80955100,No,No,No,No
13659,1.0,Yes,3.0,Yes,Two,2,3,No,No,Private,No Common Wall,33.81962200,-117.80559800,Yes,No,No,No
13660,1.0,Yes,2.0,Yes,One,3,5,Private,Yes,Private,No Common Wall,33.82315700,-117.81894700,Yes,No,No,No


In [215]:
# Get School Details Attributes
list_= set()
school_df = pd.DataFrame(index = index)
for count, school in enumerate(df['school_details_dict']):
    dict_ = ast.literal_eval(school)
    for key, value in dict_.items():
        name, rating, school_type, school_dist = [i for i in value]
        if rating:
            column = key + '_rating'
            school_df.loc[count, column] = rating.replace('NR','')
        if school_type:
            column = key + '_type'
            school_df.loc[count, column] = school_type.replace('NR','')
        if school_dist:
            column = key + '_dist'
            school_df.loc[count, column] = school_dist.replace('NR','').replace('mi','').strip()
school_df

,K to 5_rating,K to 5_type,K to 5_dist,6 to 8_rating,6 to 8_type,6 to 8_dist,9 to 12_rating,9 to 12_type,9 to 12_dist,K to 6_rating,...,5 to 8_dist,K to 3_rating,K to 3_type,K to 3_dist,K to 2_rating,K to 2_type,K to 2_dist,3 to 5_rating,3 to 5_type,3 to 5_dist
0,8,Public,0.4,7,Public,0.4,7,Public,0.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,4,Public,0.8,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,Public,1.0,7,Public,0.3,8,Public,4.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,7,Public,1.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13657,NaN,NaN,NaN,NaN,NaN,NaN,7,Public,0.7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13658,NaN,NaN,NaN,NaN,NaN,NaN,7,Public,0.7,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13659,NaN,NaN,NaN,NaN,NaN,NaN,7,Public,0.7,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13660,NaN,NaN,NaN,NaN,NaN,NaN,7,Public,0.4,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
# Get School Details Attributes
school_grade_dict = {'12':'Twelve', '3':'Three', '4':'Four', '5':'Five', 
                     '6':'Six', '7':'Seven', '8':'Eight', '9':'Nine', '10':'Ten','1':'One','2':'Two'}
school_df = pd.DataFrame(index = index)
for count, school in enumerate(df['school_details_dict']):
    dict_ = ast.literal_eval(school)
    for key, value in dict_.items():
        for grade, new_grade in school_grade_dict.items():
            key = key.replace(grade, new_grade)
            key = key.replace(' ','')
        name, rating, school_type, school_dist = [i for i in value]
        if rating:
            column = key + 'Rating'
            school_df.loc[count, column] = rating.replace('NR','')
        if school_type:
            column = key + 'Type'
            school_df.loc[count, column] = school_type.replace('NR','')
        if school_dist:
            column = key + 'Dist'
            school_df.loc[count, column] = school_dist.replace('NR','').replace(' mi','')
school_df.columns

# Check missing
for col in school_df.columns:
    percent_missing = np.sum(school_df[col].isna())/len(school_df[col])
    if percent_missing>0.90:
        print('Dropped Column:', col, '\n',
              'Percent Missing:', round(percent_missing * 100,  2))
        school_df = school_df.drop(col, axis=1)

Dropped Column: PreschooltoTwoRating 
 Percent Missing: 99.0
Dropped Column: PreschooltoTwoType 
 Percent Missing: 99.0
Dropped Column: PreschooltoTwoDist 
 Percent Missing: 99.0
Dropped Column: ThreetoSixRating 
 Percent Missing: 99.0
Dropped Column: ThreetoSixType 
 Percent Missing: 99.0
Dropped Column: ThreetoSixDist 
 Percent Missing: 99.0
Dropped Column: PreschooltoSixRating 
 Percent Missing: 93.33
Dropped Column: PreschooltoSixType 
 Percent Missing: 93.33
Dropped Column: PreschooltoSixDist 
 Percent Missing: 93.33
Dropped Column: PreschooltoFiveRating 
 Percent Missing: 95.17
Dropped Column: PreschooltoFiveType 
 Percent Missing: 95.17
Dropped Column: PreschooltoFiveDist 
 Percent Missing: 95.17
Dropped Column: SeventoTwelveRating 
 Percent Missing: 95.02
Dropped Column: SeventoTwelveType 
 Percent Missing: 95.02
Dropped Column: SeventoTwelveDist 
 Percent Missing: 95.02
Dropped Column: KtoEightRating 
 Percent Missing: 93.47
Dropped Column: KtoEightType 
 Percent Missing: 93.4

In [217]:
# Scores
scores_df = pd.DataFrame(index = index)
for count, score in enumerate(df['scores_dict']):
    dict_  = ast.literal_eval(score)
    for key, value in dict_.items():
        key = [str(i).capitalize() for i in str(key).split('_')]
        key = ''.join(key)
        if value != None:
            scores_df.loc[count, key] = value
        else:
            value = np.nan
scores_df.drop('CompetitiveScore', axis=1, inplace=True)

In [218]:
# Merge data
data = stats_df.merge(key_df, left_index = True, right_index=True, how='inner')
data = data.merge(prop_df, left_index = True, right_index=True, how='inner')
data = data.merge(school_df, left_index = True, right_index=True, how='inner')
data = data.merge(scores_df, left_index = True, right_index=True, how='inner')
for col in data.columns:
    data[col] = pd.to_numeric(data[col], errors='ignore')

In [219]:
# Save
data.to_csv('../data/redfin_cleaned.csv', index=False)